In [ ]:
import pandas as pd
import numpy as np
import random
import os

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12759 entries, 0 to 12758
Data columns (total 83 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID                               12759 non-null  object 
 1   number_of_elements               12759 non-null  int64  
 2   mean_atomic_mass                 12759 non-null  float64
 3   wtd_mean_atomic_mass             12759 non-null  float64
 4   gmean_atomic_mass                12759 non-null  float64
 5   wtd_gmean_atomic_mass            12759 non-null  float64
 6   entropy_atomic_mass              12759 non-null  float64
 7   wtd_entropy_atomic_mass          12759 non-null  float64
 8   range_atomic_mass                12759 non-null  float64
 9   wtd_range_atomic_mass            12759 non-null  float64
 10  std_atomic_mass                  12759 non-null  float64
 11  wtd_std_atomic_mass              12759 non-null  float64
 12  mean_fie          

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

학습에 불필요한 컬럼 제거

In [ ]:
X = train.drop(['ID', 'critical_temp'], axis = 1)
Y = train['critical_temp']

X_test = test.drop('ID', axis = 1)

평가 지표 정의

In [ ]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [ ]:
!pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 56.0 MB/s eta 0:0

In [ ]:
from pycaret import regression

In [ ]:
regression.setup(data=X, target=Y)

,Description,Value
0,Session id,559
1,Target,critical_temp
2,Target type,Regression
3,Original data shape,"(12759, 82)"
4,Transformed data shape,"(12759, 82)"
5,Transformed train set shape,"(8931, 82)"
6,Transformed test set shape,"(3828, 82)"
7,Numeric features,81
8,Preprocess,True
9,Imputation type,simple


pycaret 사용하여 성능이 제일 좋은 모델 1개 선택

In [ ]:
best2 = regression.compare_models(n_select=1, verbose=False, turbo=True, fold=10, sort='mae')

In [ ]:
best2

ExtraTreesRegressor(n_jobs=-1, random_state=559)

In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

앞서 pycaret에서 가장 우수했던 ExtraTreesRegressor를 optuna 써서 하이퍼 파라미터 찾음: 100회가 넘도록 변동이 없어서 중단

In [ ]:
# 데이터 로드 및 전처리 (X_train, X_valid, y_train, y_valid)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 200, 500, 10)
    max_depth = trial.suggest_int('max_depth', 40, 50, 1)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10, 1)
    bootstrap = trial.suggest_categorical('bootstrap', [False])


    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
         min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=bootstrap,
        random_state=559
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = NMAE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2023-08-19 07:33:14,088] A new study created in memory with name: no-name-da583d11-85b5-414c-b38f-97f3e8aba8ee
[I 2023-08-19 07:33:55,700] Trial 0 finished with value: 0.15396392527574923 and parameters: {'n_estimators': 270, 'max_depth': 41, 'min_samples_split': 4, 'bootstrap': False}. Best is trial 0 with value: 0.15396392527574923.
[I 2023-08-19 07:35:00,036] Trial 1 finished with value: 0.15729016443596222 and parameters: {'n_estimators': 460, 'max_depth': 40, 'min_samples_split': 8, 'bootstrap': False}. Best is trial 0 with value: 0.15396392527574923.
[I 2023-08-19 07:35:36,210] Trial 2 finished with value: 0.15640335197615304 and parameters: {'n_estimators': 270, 'max_depth': 50, 'min_samples_split': 7, 'bootstrap': False}. Best is trial 0 with value: 0.15396392527574923.
[I 2023-08-19 07:36:22,858] Trial 3 finished with value: 0.15740514690153704 and parameters: {'n_estimators': 360, 'max_depth': 46, 'min_samples_split': 8, 'bootstrap': False}. Best is trial 0 with value: 0.1

KeyboardInterrupt: ignored

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

Best trial: {'n_estimators': 480, 'max_depth': 48, 'min_samples_split': 2, 'bootstrap': False}
Best score: 0.15380567982855284


In [ ]:
params ={'n_estimators': 480, 'max_depth': 48, 'min_samples_split': 2, 'bootstrap': False}
model = ExtraTreesRegressor(**params)
model.fit(X, Y)

# 테스트 데이터에 대한 예측
preds = model.predict(X_test)

submission3 = pd.read_csv('sample_submission.csv')
submission3['critical_temp'] = preds
submission3.head()

,ID,critical_temp
0,TEST_00000,1.645000
1,TEST_00001,4.372921
2,TEST_00002,12.756827
3,TEST_00003,46.365840
4,TEST_00004,3.903477


In [ ]:
submission3.to_csv('./optuna.csv', index=False)

In [ ]:
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

autogluon 5시간 적용

In [ ]:
train_data = TabularDataset('/content/train.csv')
test_data = TabularDataset('/content/test.csv')

label = 'critical_temp'
eval_metric = 'mae'
time_limit = 3600 * 5

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, num_gpus=1)
predictor.leaderboard(silent = True)

No path specified. Models will be saved in: "AutogluonModels/ag-20230820_015418/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 18000s
AutoGluon will save models to "AutogluonModels/ag-20230820_015418/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   144.59 GB / 179.07 GB (80.7%)
Train Data Rows:    12759
Train Data Columns: 82
Label Column: critical_temp
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (185.0, 0.00032, 34.40876, 34.24424)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify prob

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-4.922891,186.376625,14257.072970,0.000815,0.701731,3,True,22
1,ExtraTreesMSE_BAG_L2,-4.937110,132.486308,8809.828918,1.030174,63.857607,2,True,17
2,RandomForestMSE_BAG_L2,-4.982041,132.501813,9118.438770,1.045678,372.467459,2,True,15
3,LightGBMLarge_BAG_L2,-5.041986,181.909775,12710.188458,50.453640,3964.217147,2,True,21
4,WeightedEnsemble_L2,-5.192567,92.115633,3877.439284,0.001045,0.791831,2,True,12
5,XGBoost_BAG_L2,-5.243913,133.486714,8912.295091,2.030580,166.323780,2,True,19
6,LightGBMXT_BAG_L1,-5.259838,75.402758,1459.364995,75.402758,1459.364995,1,True,3
7,LightGBM_BAG_L2,-5.275559,133.410788,9089.575603,1.954653,343.604292,2,True,14
8,LightGBMLarge_BAG_L1,-5.316213,14.684749,1110.639771,14.684749,1110.639771,1,True,11
9,NeuralNetTorch_BAG_L2,-5.317844,133.846318,9855.829025,2.390183,1109.857714,2,True,20


In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)
submission5 = pd.read_csv('sample_submission.csv')
submission5['critical_temp'] = model_pred
submission5.head()

,ID,critical_temp
0,TEST_00000,1.762496
1,TEST_00001,4.591799
2,TEST_00002,12.274580
3,TEST_00003,43.019215
4,TEST_00004,4.089535


In [ ]:
submission5.to_csv('./autogluon5.csv', index=False)

In [ ]:
ag = pd.read_csv('/content/autogluon5.csv')
ot = pd.read_csv('/content/optuna.csv')

autogluon 예측값과 optuna 예측값 앙상블

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['critical_temp'] = (ag['critical_temp'] + ot['critical_temp']) / 2
submission.to_csv('./ag_ot.csv', index=False)